# Prepare the setup

In [ ]:
# Import packages
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay

In [2]:
# Load the dataset
df = pd.read_csv("../data/new_training_dataset.csv")

NameError: name 'pd' is not defined

This dataset was already in csv format, so we imported it directly. To prepare it we removed some columns we didn't need for the model development. Most of those were categorical type which we didn't need. 


# Initial Exploration

In [1]:
# See the first 5 rows
df.head()

NameError: name 'df' is not defined

In [ ]:
# Print information about data
df.info()

In [ ]:
df.describe()

In [ ]:
# Check for null values
df.isna().sum()

In [ ]:
# Check for duplicates
df.duplicated().sum()

In [ ]:
# Check target class
df['label'].value_counts()